In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For Pre-Processing
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

# For GNN

import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl
import dgl.function as fn
from dgl.nn import GraphConv

import networkx as nx
from networkx.algorithms import bipartite


In [56]:
ratings = pd.read_csv("Data/Ratings.csv")
books = pd.read_csv("Data/Books.csv")
users = pd.read_csv("Data/Users.csv")

/var/folders/0f/xqjk7pr56mjd12xvy1w435_40000gn/T/ipykernel_79395/3661012832.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("Data/Books.csv")


In [57]:
def extract_country(location):
    # Split the location by comma and strip whitespace
    parts = [part.strip() for part in location.split(',')]
    # Return the last part as the country
    return parts[-1] if parts else None

users['Country'] = users['Location'].apply(extract_country)

# knn_imputer = KNNImputer(n_neighbors=5)
# users['Age'] = knn_imputer.fit_transform(users[['Age']])

In [61]:
ratings['Book-Title'] = ratings['Book-Title'].replace(to_replace='\$', value='', regex=True)
ratings['Book-Author'] = ratings['Book-Author'].replace(to_replace='\$', value='', regex=True)

books_and_ratings = ratings.merge(books, on='ISBN')
books_and_ratings = books_and_ratings.drop(columns=["Image-URL-S", "Image-URL-M", "Image-URL-L"])
books_and_ratings = books_and_ratings[books_and_ratings['Book-Rating'] > 0]

books_and_ratings.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press
6,276744,038550120X,7,A Painted House,JOHN GRISHAM,2001,Doubleday
13,276747,0060517794,9,Little Altars Everywhere,Rebecca Wells,2003,HarperTorch


In [60]:
user_node = ratings['User-ID'].unique()
book_node = ratings['Book-Title'].unique()

KeyError: 'Book-Title'

In [ ]:
ratings = ratings[["User-ID","Book-Title","Book-Rating","ISBN"]]
ratings100 = ratings.head(20) 
ratings100

In [ ]:
G = nx.Graph()
user_nodes=list(ratings100['User-ID'])
book_nodes=list(ratings100['Book-Title'])
# Add nodes with the node attribute "bipartite"

G.add_nodes_from(user_nodes, bipartite=0)
G.add_nodes_from(book_nodes, bipartite=1)


weights = []
for index, row in ratings100.iterrows():
    G.add_edge(row["User-ID"], row["Book-Title"], weight=row["Book-Rating"])
    weights.append(row["Book-Rating"])
bipartite.is_bipartite(G)

zip_list=list(zip(user_nodes,book_nodes))
dict_of_labels = dict()
for i in range(len(zip_list)):
    dict_of_labels[zip_list[i]]=weights[i]
    
plt.figure(figsize=(15, 10))

pos = nx.bipartite_layout(G, user_nodes)

# Draw the graph
nx.draw_networkx_nodes(G, pos, nodelist=user_nodes, node_color='blue', node_shape='o', label='users', node_size=500)
nx.draw_networkx_nodes(G, pos, nodelist=book_nodes, node_color='green', node_shape='^', label='books',node_size=500)
nx.draw_networkx_edges(G, pos, edge_color='gray', width=[d['weight']*0.1 for (u, v, d) in G.edges(data=True)])

nx.draw_networkx_edge_labels(G, pos, edge_labels=dict_of_labels, font_size=20)
nx.draw_networkx_labels(G, pos)

plt.legend()
plt.show()

In [ ]:
lbl_user = LabelEncoder()
lbl_movie = LabelEncoder()

ratings["User-ID"] = lbl_user.fit_transform(ratings["User-ID"].values)
ratings["ISBN"] = lbl_movie.fit_transform(ratings.ISBN.values)

In [ ]:
ratings.head()